## 哈希表/集合

### 哈希函数

1. 输入无穷 输出有穷
2. same input -> same ouput
3. diffrence input -> same output 即哈希碰撞
4. 

##### 哈希冲突

1. 开放寻址法

开放寻址法的核心思想是，如果出现了散列冲突，我们就重新探测一个空闲位置，将其插入。

2. 链表法

链表法是一种更加常用的散列冲突解决方法，相比开放寻址法，它要简单很多。如图，在散列表中，每个“桶（bucket）”或者“槽（slot）”会对应一条链表，所有散列值相同的元素我们都放到相同槽位对应的链表中。

![image.png](./images/a4b77d593e4cb76acb2b0689294ec17f.png)

当插入的时候，我们只需要通过散列函数计算出对应的散列槽位，将其插入到对应链表中即可，所以插入的时间复杂度是 O(1)。当查找、删除一个元素时，我们同样通过散列函数计算出对应的槽，然后遍历链表查找或者删除。

#### 题目

1. 设计集合

In [2]:
class HashSet:

    def __init__(self):
        self.data = [[], ] * 769
    
    def _hash(self, key):
        return key % 769

    def add(self, key: int) -> None:
        index = self._hash(key)
        for i in range(len(self.data[index])):
            if self.data[index][i] == key:
                return
        self.data[index].append(key)

    def remove(self, key: int) -> None:
        index = self._hash(key)
        for i in range(len(self.data[index])):
            if key == self.data[index][i]:
                self.data[index].pop(i)
                return

    def contains(self, key: int) -> bool:
        index = self._hash(key)
        for i in range(len(self.data[index])):
            if self.data[index][i] == key:
                return True
        return False

2. 设计 RandomPool 结构

设计一种结构，在该结构中有如下三个功能：

- insert(key)：将某个 key 加入到该结构中，做到不重复加入
- delete(key)：将原本在结构中的某个 key 移除
- getRandom()：等概率随机返回结构中的任何一个 key

要求 insert, delete, getRandom 的时间复杂度都是 O(1)

In [17]:
class RandomPool:

    def __init__(self):
        self.key2index = {}
        self.index2key = {}
        self.size = 0

    def insert(self, key) -> None:
        if self.key2index.get(key) == None:
            self.key2index[key] = self.size
            self.index2key[self.size] = key
            self.size += 1
        return

    def delete(self, key) -> None:
        
        if key not in self.key2index:
            return
        if self.size == 1:
            self.key2index.pop(self.index2key[0])
            self.index2key.pop(0)
            return
        index = self.key2index.get(key)
        lastKey = self.index2key[self.size - 1]
        self.key2index[lastKey] = index
        self.index2key[index] = lastKey
        self.index2key.pop(self.size - 1)
        self.key2index.pop(key)
        return

    def getRandom(self):
        if self.size == 0:
            return None
        import random
        index = random.randint(0, self.size - 1)
        return self.index2key.get(index)

## 布隆过滤器

$$m=-\frac{n*ln{p}}{(ln{2})^{2}}$$


$$k=ln{2}*\frac{m}{n}$$

$$p_{1}=(1-e^{-\frac{n*k_{1}}{m_{1}}})^{k_{1}}$$

其中：

m 是 bitarray 的大小，n 是样本数量，p 是预计失误率，k 是哈希函数的个数

In [135]:
class BloomFilter:
    """位图是特殊的布隆过滤器
    """

    def __init__(self, n=10, p=0.1):
        import math
        from hashlib import md5, sha1, sha224, sha512
        self.n = n
        self.m = int(-n * math.log(p) / (math.log(2) ** 2)) + 1
        self.k = int(math.log(2) * self.m / n) + 1
        self.bitarray = bytearray(self.m // 8 + 1)
        self._hash = array = [md5, sha1, sha224, sha512]
    
    def _set(self, i, j):
        self.bitarray[i] |= 1 << j
    
    def _get(self, i, j):
        return self.bitarray[i] & (1 << j)
    
    def add(self, item: str):
        for k in range(self.k):
            digest = int(self._hash[k](item.encode()).hexdigest(), 16) % self.m
            i, j = digest // 8, digest % 8 
            self._set(i, j)

    def check(self, item: str):
        for k in range(self.k):
            digest = int(self._hash[k](item.encode()).hexdigest(), 16) % self.m
            i, j = digest // 8, digest % 8
            return self._get(i, j) != 0

## 一致性哈希原理